In [1]:
import numpy as np
import skallel_tensor
from skallel_tensor import genotypes_count_alleles
from numba import cuda
skallel_tensor.__version__

'0.1.0a5.dev0+dirty'

# Smaller dataset (400MB)

In [2]:
# simulate some genotype data
gt = np.random.choice(np.array([-1, 0, 1, 2, 3], dtype='i1'), 
                      p=[.01, .7, .2, .08, .01], 
                      size=(200_000, 1_000, 2))

In [3]:
gt.nbytes / 1e6

400.0

In [4]:
gt.dtype

dtype('int8')

In [5]:
gt

array([[[0, 0],
        [1, 0],
        [0, 0],
        ...,
        [0, 0],
        [2, 0],
        [0, 0]],

       [[0, 2],
        [0, 1],
        [0, 1],
        ...,
        [1, 0],
        [2, 0],
        [2, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [2, 0],
        [0, 0],
        [1, 0]],

       ...,

       [[0, 0],
        [0, 0],
        [1, 0],
        ...,
        [0, 0],
        [1, 1],
        [0, 0]],

       [[0, 0],
        [0, 1],
        [1, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 2]],

       [[0, 1],
        [0, 1],
        [1, 0],
        ...,
        [0, 0],
        [1, 1],
        [0, 0]]], dtype=int8)

In [6]:
gt.shape

(200000, 1000, 2)

## Single CPU core

In [7]:
# warm-up jit
genotypes_count_alleles(gt, max_allele=3)

array([[1394,  426,  142,   18],
       [1424,  390,  148,   23],
       [1444,  383,  135,   21],
       ...,
       [1377,  426,  166,   17],
       [1429,  383,  149,   23],
       [1414,  388,  145,   31]], dtype=int32)

In [8]:
%%timeit -n1 -r3
genotypes_count_alleles(gt, max_allele=3)

456 ms ± 3.16 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## Multiple CPU cores via Dask

In [9]:
import dask.array as da

In [10]:
gt_dask = da.from_array(gt, chunks=(10_000, None, None))
gt_dask

dask.array<array, shape=(200000, 1000, 2), dtype=int8, chunksize=(10000, 1000, 2)>

In [11]:
%%timeit -n1 -r5
genotypes_count_alleles(gt_dask, max_allele=3).compute()

152 ms ± 57.8 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


## Single GPU

In [12]:
!nvidia-smi | head

Mon Jul 22 13:02:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro M1000M       Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0    N/A /  N/A |    933MiB /  2004MiB |      5%      Default |
+-------------------------------+----------------------+----------------------+


In [13]:
%%time
gt_cuda = cuda.to_device(gt)
gt_cuda

CPU times: user 70 ms, sys: 104 ms, total: 174 ms
Wall time: 173 ms


In [14]:
# warm up jit
genotypes_count_alleles(gt_cuda, max_allele=3).copy_to_host()

array([[1394,  426,  142,   18],
       [1424,  390,  148,   23],
       [1444,  383,  135,   21],
       ...,
       [1377,  426,  166,   17],
       [1429,  383,  149,   23],
       [1414,  388,  145,   31]], dtype=int32)

In [15]:
%%timeit -n1 -r5
genotypes_count_alleles(gt_cuda, max_allele=3)
cuda.synchronize()

181 ms ± 9.86 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [16]:
ac_cuda = genotypes_count_alleles(gt_cuda, max_allele=3)
ac_cuda

In [17]:
%time ac_cuda.copy_to_host()

CPU times: user 149 ms, sys: 40.2 ms, total: 189 ms
Wall time: 189 ms


array([[1394,  426,  142,   18],
       [1424,  390,  148,   23],
       [1444,  383,  135,   21],
       ...,
       [1377,  426,  166,   17],
       [1429,  383,  149,   23],
       [1414,  388,  145,   31]], dtype=int32)

## GPU via Dask

In [18]:
gt_dask_cuda = gt_dask.map_blocks(cuda.to_device)

In [19]:
genotypes_count_alleles(gt_dask_cuda, max_allele=3).compute()

array([[1394,  426,  142,   18],
       [1424,  390,  148,   23],
       [1444,  383,  135,   21],
       ...,
       [1377,  426,  166,   17],
       [1429,  383,  149,   23],
       [1414,  388,  145,   31]], dtype=int32)

In [20]:
%%timeit -n1 -r3
genotypes_count_alleles(gt_dask_cuda, max_allele=3).compute()

403 ms ± 15.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


# Larger dataset (4GB)

In [21]:
# gt_random_big = da.random.choice(
#     np.array([-1, 0, 1, 2, 3], dtype='i1'), 
#     p=[.01, .7, .2, .08, .01], 
#     size=(2_000_000, 1_000, 2),
#     chunks=(50_000, None, None))
# gt_random_big

In [22]:
# gt_random_big.to_zarr('example.zarr', component='gt')

In [23]:
import zarr
gt_big_zarr = zarr.open('example.zarr')['gt']
gt_big_zarr.info

Name,/gt
Type,zarr.core.Array
Data type,int8
Shape,"(2000000, 1000, 2)"
Chunk shape,"(50000, 1000, 2)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,4000000000 (3.7G)
No. bytes stored,2297009415 (2.1G)


In [24]:
gt_big_dask = da.from_array(gt_big_zarr)
gt_big_dask

,Array,Chunk
Bytes,4.00 GB,100.00 MB
Shape,"(2000000, 1000, 2)","(50000, 1000, 2)"
Count,41 Tasks,40 Chunks
Type,int8,numpy.ndarray


## Multiple CPUs via Dask

In [25]:
%%time
genotypes_count_alleles(gt_big_dask, max_allele=3).compute()

CPU times: user 13.4 s, sys: 3.12 s, total: 16.5 s
Wall time: 2.33 s


array([[1420,  390,  150,   18],
       [1367,  432,  166,   17],
       [1370,  423,  168,   20],
       ...,
       [1366,  426,  173,   19],
       [1376,  423,  173,   15],
       [1392,  411,  156,   23]], dtype=int32)

## Single GPU via Dask

In [26]:
gt_big_dask_cuda = gt_big_dask.map_blocks(cuda.to_device)

In [27]:
%%time
genotypes_count_alleles(gt_big_dask_cuda, max_allele=3).compute(num_workers=1)

CPU times: user 6.09 s, sys: 2.9 s, total: 9 s
Wall time: 9 s


array([[1420,  390,  150,   18],
       [1367,  432,  166,   17],
       [1370,  423,  168,   20],
       ...,
       [1366,  426,  173,   19],
       [1376,  423,  173,   15],
       [1392,  411,  156,   23]], dtype=int32)

In [28]:
%%time
genotypes_count_alleles(gt_big_dask_cuda, max_allele=3).compute(num_workers=2)

CPU times: user 6.15 s, sys: 3.23 s, total: 9.38 s
Wall time: 5.21 s


array([[1420,  390,  150,   18],
       [1367,  432,  166,   17],
       [1370,  423,  168,   20],
       ...,
       [1366,  426,  173,   19],
       [1376,  423,  173,   15],
       [1392,  411,  156,   23]], dtype=int32)

In [29]:
%%time
genotypes_count_alleles(gt_big_dask_cuda, max_allele=3).compute(num_workers=3)

CPU times: user 6.55 s, sys: 3.32 s, total: 9.87 s
Wall time: 3.79 s


array([[1420,  390,  150,   18],
       [1367,  432,  166,   17],
       [1370,  423,  168,   20],
       ...,
       [1366,  426,  173,   19],
       [1376,  423,  173,   15],
       [1392,  411,  156,   23]], dtype=int32)

In [30]:
%%time
genotypes_count_alleles(gt_big_dask_cuda, max_allele=3).compute(num_workers=4)

CPU times: user 6.68 s, sys: 3.45 s, total: 10.1 s
Wall time: 3.93 s


array([[1420,  390,  150,   18],
       [1367,  432,  166,   17],
       [1370,  423,  168,   20],
       ...,
       [1366,  426,  173,   19],
       [1376,  423,  173,   15],
       [1392,  411,  156,   23]], dtype=int32)

Interesting thing, even with just a single GPU, we can improve performance and get better saturation by having multiple Dask workers.